In [13]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from transformers import AutoModelForSequenceClassification, AutoTokenizer, BertConfig, BertForSequenceClassification
from datasets import load_dataset
from tqdm import tqdm
from copy import deepcopy
import torch.nn.functional as F

model_id ="prajjwal1/bert-tiny"
model = AutoModelForSequenceClassification.from_pretrained(model_id, num_labels=2, use_safetensors=True )
tokenizer = AutoTokenizer.from_pretrained(model_id)
print(model)
#

ValueError: Unrecognized model in prajjwal1/bert-tiny. Should have a `model_type` key in its config.json.

In [2]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from transformers import AutoModelForSequenceClassification, AutoTokenizer, BertConfig, BertForSequenceClassification
from datasets import load_dataset
from tqdm import tqdm
from copy import deepcopy
import torch.nn.functional as F

model_id ="prajjwal1/bert-tiny"
# model = AutoModelForSequenceClassification.from_pretrained(model_id, num_labels=2, use_safetensors=True )
# tokenizer = AutoTokenizer.from_pretrained(model_id)
#


config = BertConfig.from_pretrained(model_id, num_labels=2)
model = BertForSequenceClassification.from_pretrained(model_id, config=config)
print( model)



/home/latim/PycharmProjects/WatDNN/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading weights: 100%|██████████| 39/39 [00:00<00:00, 1893.04it/s, Materializing param=bert.pooler.dense.weight]                              
BertForSequenceClassification LOAD REPORT from: prajjwal1/bert-tiny
Key                                        | Status     | 
-------------------------------------------+------------+-
cls.predictions.decoder.weight             | UNEXPECTED | 
cls.seq_relationship.weight                | UNEXPECTED | 
cls.predictions.transform.dense.weight     | UNEXPECTED | 
cls.predictions.transform.LayerNorm.bias   | UNEXPECTED | 
bert.embeddings.position_ids               | UNEXPECTED | 
cls.predictions.decoder.bias               | UNEXPECTED | 
cls.predictions.transform.dense.bias       

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-1): 2 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=128, out_features=128, bias=True)
              (key): Linear(in_features=128, out_features=128, bias=True)
              (value): Linear(in_features=128, out_features=128, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=128, out_features=128, bias=True)
              (LayerNorm): LayerNorm((128,), eps=1e-12, e

In [8]:
for name, param in model.named_parameters():
            print(f"  {name}: {param.shape}")
            if name == "bert.encoder.layer.1.output.dense.weight":
                print("  This is the word embedding layer.")

print("*******************************")
for name, param in model.named_modules():
            if name == "bert.encoder.layer.1.output.dense":
                print("  This is the word embedding layer.")
            print(f"  {name}: {param}")
            if name == "bert.encoder.layer.1.output":
                print("  This is the word embedding layer.")


  bert.embeddings.word_embeddings.weight: torch.Size([30522, 128])
  bert.embeddings.position_embeddings.weight: torch.Size([512, 128])
  bert.embeddings.token_type_embeddings.weight: torch.Size([2, 128])
  bert.embeddings.LayerNorm.weight: torch.Size([128])
  bert.embeddings.LayerNorm.bias: torch.Size([128])
  bert.encoder.layer.0.attention.self.query.weight: torch.Size([128, 128])
  bert.encoder.layer.0.attention.self.query.bias: torch.Size([128])
  bert.encoder.layer.0.attention.self.key.weight: torch.Size([128, 128])
  bert.encoder.layer.0.attention.self.key.bias: torch.Size([128])
  bert.encoder.layer.0.attention.self.value.weight: torch.Size([128, 128])
  bert.encoder.layer.0.attention.self.value.bias: torch.Size([128])
  bert.encoder.layer.0.attention.output.dense.weight: torch.Size([128, 128])
  bert.encoder.layer.0.attention.output.dense.bias: torch.Size([128])
  bert.encoder.layer.0.attention.output.LayerNorm.weight: torch.Size([128])
  bert.encoder.layer.0.attention.output.L

In [16]:
dataset = load_dataset("glue", "sst2")

In [17]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 67349
    })
    validation: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 872
    })
    test: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 1821
    })
})


In [18]:
print(dataset['train'][0])

{'sentence': 'hide new secretions from the parental units ', 'label': 0, 'idx': 0}


In [19]:
import inspect
print(inspect.signature(model.forward))

(input_ids: torch.Tensor | None = None, attention_mask: torch.Tensor | None = None, token_type_ids: torch.Tensor | None = None, position_ids: torch.Tensor | None = None, inputs_embeds: torch.Tensor | None = None, labels: torch.Tensor | None = None, **kwargs: Unpack[transformers.utils.generic.TransformersKwargs]) -> tuple[torch.Tensor] | transformers.modeling_outputs.SequenceClassifierOutput


In [21]:
print(model.config)

BertConfig {
  "add_cross_attention": false,
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": null,
  "classifier_dropout": null,
  "dtype": "float32",
  "eos_token_id": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 128,
  "initializer_range": 0.02,
  "intermediate_size": 512,
  "is_decoder": false,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 2,
  "num_hidden_layers": 2,
  "pad_token_id": 0,
  "tie_word_embeddings": true,
  "transformers_version": "5.2.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

